In [1]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

model = torchvision.models.mobilenet_v2(pretrained=True)        # 사전학습된 모델 가져옴
model.eval()
example = torch.rand(1, 3, 224, 224)                            # 모델 사이즈에 맞는 Tensor 생성
traced_script_module = torch.jit.trace(model, example)          
traced_script_module_optimized = optimize_for_mobile(traced_script_module)      
traced_script_module_optimized._save_for_lite_interpreter("app/src/main/assets/model.ptl")  # 해당 코드 실행하면 app/src/main/assets dir에 model.ptl 생성됨

- 생성된 가중치파일이 Android app에서 asset으로 작동함  

![](/images/2022-02-18-14-09-23.png)

- build.gradle 의존성 셋팅

  ```bash
  repositories {
      jcenter()
  }

  dependencies {
      implementation 'org.pytorch:pytorch_android_lite:1.9.0'     // Pytorch Android API
      implementation 'org.pytorch:pytorch_android_torchvision:1.9.0'      // android.media.Image -> android.graphics.Bitmap으로 바꿔주는 기능
  }
  ```

- png, jpg 이미지 안드로이드 Bitmap 객체로 변환
  
  ```java
  Bitmap bitmap = BitmapFactory.decodeStream(getAssets().open("image.jpg"));
  ```

- **가중치 파일** 로드

  ```java
  Module module = Module.load(assetFilePath(this, "model.ptl"));
  ```

- Bitmap 객체 32bit float Tensor로 변환 및 전처리
  - 주의사항으로 로드된 가중치파일이 학습할 때 아래 코드와 같은 `전처리`가 이루어진것으로 해야 성능 하락을 억제할 수 있음
    ```java
    Tensor inputTensor = TensorImageUtils.bitmapToFloat32Tensor(bitmap,
        TensorImageUtils.TORCHVISION_NORM_MEAN_RGB, TensorImageUtils.TORCHVISION_NORM_STD_RGB);
        // mean: ([0.485, 0.456, 0.406]) std: ([0.229, 0.224, 0.225])임. 얘네가 Imagenet 전체 데이터에 계산한 평균, 표준편차 값임
        // 이미지 사이즈는 자동으로 정해주나..?
    ```

- Inference

  ```java
  Tensor outputTensor = module.forward(IValue.from(inputTensor)).toTensor();
  float[] scores = outputTensor.getDataAsFloatArray();        // pretrained된 imagenet으로 진행했다면 (1, 1000)으로 output 나옴

  float maxScore = -Float.MAX_VALUE;      
  int maxScoreIdx = -1;
  for (int i = 0; i < scores.length; i++) {       // Inference 값 중 가장 큰 값의 index 구함
    if (scores[i] > maxScore) {
      maxScore = scores[i];
      maxScoreIdx = i;
    }
  }
  String className = ImageNetClasses.IMAGENET_CLASSES[maxScoreIdx];
  ```

- 의문사항
  - 왜 img, ptl을 assets 폴더에 두는거지?
  - 평균이 0.5가 아니라 저렇게 나오는 이유가 있나..? 표준 정규화 거치면 무조건 평균 0, 표준편차 1로 되는거 아님?

In [2]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile
from ast import Num
import torch
import numpy as np
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import os
import math
from glob import glob
from PIL import Image
import pandas as pd

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.classifier[1] = nn.Linear(in_features=1280,out_features=7,bias=True)
model.eval()
example = torch.rand(1, 3, 252, 336)
traced_script_module = torch.jit.trace(model, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("app/src/main/assets/model_custom.ptl")

Using cache found in C:\Users\hojun_window/.cache\torch\hub\pytorch_vision_v0.10.0


In [1]:
import torch

# use deeplabv3_resnet50 instead of resnet101 to reduce the model size
model = torch.hub.load('pytorch/vision:v0.7.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

scriptedm = torch.jit.script(model)
torch.jit.save(scriptedm, "deeplabv3_scripted.pt")

Downloading: "https://github.com/pytorch/vision/archive/v0.7.0.zip" to C:\Users\hojun_window/.cache\torch\hub\v0.7.0.zip
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\hojun_window/.cache\torch\hub\checkpoints\resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:08<00:00, 11.7MB/s]
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to C:\Users\hojun_window/.cache\torch\hub\checkpoints\deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:14<00:00, 11.6MB/s] 
c:\Users\hojun_window\anaconda3\envs\yolov5\lib\site-packages\torch\jit\_recursive.py:234: UserWarning: 'aux_classifier' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn("'{}' was found in ScriptModule constants, "
